# Data Ingestion

# working with CSV 

In [ ]:
import pandas as pd

df = pd.read_csv("taxi.csv")
# print("Columns Type Before -\n",df.dtypes)


time_col = ['tpep_pickup_datetime','tpep_dropoff_datetime']
df = pd.read_csv("taxi.csv",parse_dates=time_col)
# print("\nColumns Type after -\n",df.dtypes)


df_parser = pd.read_csv('taxi.csv',parse_dates=time_col,chunksize=1000)
df_parser
for df in df_parser:
    print(len(df))

# working with xml

In [ ]:
import bz2
import xml.etree.ElementTree as xml

import pandas as pd

# Data conversions
conversion = [
    ('vendor', int),
    ('people', int),
    ('tip', float),
    ('price', float),
    ('pickup', pd.to_datetime),
    ('dropoff', pd.to_datetime),
    ('distance', float),
]


def iter_rides(file_name):
    with bz2.open(file_name, 'rt') as fp:
        tree = xml.parse(fp)

    rides = tree.getroot()
    for elem in rides:
        record = {}
        for tag, func in conversion:
            text = elem.find(tag).text
            record[tag] = func(text)
        yield record


def load_xml(file_name):
    records = iter_rides(file_name)
    return pd.DataFrame.from_records(records)


# Example
if __name__ == '__main__':
    df = load_xml('taxi.xml.bz2')
    print(df.dtypes)
    print(df.head())

# Working with different format

Parquet , Avro and ORC

In [7]:
import pyarrow.parquet as pq

table = pq.read_table('taxi.parquet')
df = table.to_pandas()
df.dtypes
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2018-10-31 07:10:55,2018-11-01 06:43:24,1,2.57,1,N,211,48,1,14.5,0.5,0.5,4.74,0.0,0.3,20.54
1,2,2018-10-31 16:38:25,2018-10-31 16:50:10,5,3.58,1,N,237,144,2,12.5,0.5,0.5,0.00,0.0,0.3,13.80
2,2,2018-10-31 20:23:41,2018-10-31 20:31:47,1,2.39,1,N,163,107,1,9.0,0.5,0.5,1.00,0.0,0.3,11.30
3,2,2018-10-31 22:44:24,2018-10-31 22:48:28,1,0.50,1,N,246,246,2,4.5,0.5,0.5,0.00,0.0,0.3,5.80
4,2,2018-10-31 23:22:18,2018-10-31 23:35:30,1,1.81,1,N,79,90,1,10.0,0.5,0.5,2.26,0.0,0.3,13.56


# Challenge: CSV to JSON

In [55]:
import pandas as pd
import json

df = pd.read_csv('taxi.csv')

print(df.loc[0,'tpep_pickup_datetime'])

jsonmap = {}

count = 0
print(len(df))
for ds in df:
    print(ds)
    temp = []
    for i in range(0,len(df)):
        temp.append(str(df.loc[i,ds]))
    jsonmap[ds] = temp

# json.dumps(jsonmap)

with open("sample.json", "w") as outfile:
    outfile.write(json.dumps(jsonmap))


2018-10-31 07:10:55
10000
VendorID
tpep_pickup_datetime
tpep_dropoff_datetime
passenger_count
trip_distance
RatecodeID
store_and_fwd_flag
PULocationID
DOLocationID
payment_type
fare_amount
extra
mta_tax
tip_amount
tolls_amount
improvement_surcharge
total_amount


In [57]:
"""Convert CSV to JSON (one object per line)"""
import bz2
import csv
import json
from collections import namedtuple
from datetime import datetime

Column = namedtuple('Column', 'src dest convert')


def parse_timestamp(text):
    return datetime.strptime(text, '%Y-%m-%d %H:%M:%S')


columns = [
    Column('VendorID', 'vendor_id', int),
    Column('passenger_count', 'num_passengers', int),
    Column('tip_amount', 'tip', float),
    Column('total_amount', 'price', float),
    Column('tpep_dropoff_datetime', 'dropoff_time', parse_timestamp),
    Column('tpep_pickup_datetime', 'pickup_time', parse_timestamp),
    Column('trip_distance', 'distance', float),
]


def iter_records(file_name):
    with bz2.open(file_name, 'rt') as fp:
        reader = csv.DictReader(fp)
        for csv_record in reader:
            record = {}
            for col in columns:
                value = csv_record[col.src]
                record[col.dest] = col.convert(value)
            yield record


def encode_time(obj):
    if not isinstance(obj, datetime):
        return obj
    return obj.isoformat()


with open('taxi.jl', 'w') as out:
    for record in iter_records('taxi.csv.bz2'):
        data = json.dumps(record, default=encode_time)
        out.write(f'{data}\n')

# Working with JSON

In [91]:
import json
from datetime import datetime

data = {
  "from": "Wile. E. Coyote",
  "to": "ACME",
  "amount": 103.7,
  "time": "2019-08-07T12:28:39.781551"
}
for i in data:
    print(i,":",data[i])
    if(i == 'time'):
        datetime.strptime(str(data[i]),"%Y-%m-%dT%H:%M:%S.%f")
        print(type(data[i]))

from : Wile. E. Coyote
to : ACME
amount : 103.7
time : 2019-08-07T12:28:39.781551
<class 'str'>


In [92]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!
